In [ ]:
%matplotlib widget

import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import ipywidgets as widgets
from functools import partial
import networkx as nx
import json
import numpy as np

from ophys_etl.modules.segmentation.graph_utils import plotting as guplot
from ophys_etl.modules.decrosstalk.ophys_plane import OphysROI, OphysMovie
from ophys_etl.modules.decrosstalk.ophys_plane import find_overlapping_roi_pairs
from ophys_etl.modules.segmentation.qc_utils.roi_utils import ROIExaminer
from ophys_etl.modules.segmentation.qc_utils.roi_utils import add_roi_boundaries_to_img
from ophys_etl.modules.segmentation.qc_utils.roi_utils import roi_thumbnail

In [ ]:
basedir = Path("/allen/programs/braintv/workgroups/nc-ophys/danielk/deepinterpolation/experiments")
experiment_ids = [785569470, 1048483611, 1048483613, 1048483616, 785569447,
                  788422859, 795901850, 788422825, 795897800, 850517348,
                  951980473, 951980484, 795901895, 806862946, 803965468,
                  806928824]
metadata_files = [basedir / f"ophys_experiment_{i}" / "metadata.csv" for i in experiment_ids]

### Available Experiments

In [ ]:
frames = pd.concat([pd.read_csv(file, header=None).set_index(0).T for file in metadata_files]).set_index("ophys_experiment_id")
frames["depth"] = frames["depth"].astype(int)
frames = frames.sort_values(by=["code", "rig", "depth"]).reset_index()
display(frames)

In [ ]:
df_per_row = [pd.DataFrame(i[1]).T for i in frames.iterrows()]
col_widths = [max([len(f"{i}") for i in frames[c].values]) for c in frames.columns]
options = ["".join([f"{i:{w + 4}}" for i, w in zip(k.values[0], col_widths)]) for k in df_per_row]
eids = [int(i["ophys_experiment_id"].values[0]) for i in df_per_row]
values = [basedir / f"ophys_experiment_{i}" for i in eids]
experiment_selector = widgets.Dropdown(
    options=[(o, v) for o, v in zip(options, values)],
    layout={'width': 'max-content'}, # If the items' names are long
    description='Select experiment:',
    disabled=False
)
display(experiment_selector)

In [ ]:
def get_max_ave_projections(result_dir: Path):
    nmax = result_dir / "noised_maxp.png"
    navg = result_dir / "noised_avgp.png"
    dnmax = result_dir / "denoised_maxp.png"
    dnavg = result_dir / "denoised_avgp.png"

    nmaxim = np.array(Image.open(nmax))
    navgim = np.array(Image.open(navg))
    dnmaxim = np.array(Image.open(dnmax))
    dnavgim = np.array(Image.open(dnavg))

    return nmaxim, dnmaxim, navgim, dnavgim

In [ ]:
def new_background_selector(nrows, ncols):
    available = []
    for ext in ["png", "pkl"]:
        available.extend(list((experiment_selector.value / "backgrounds").rglob(f"*.{ext}")))
    background_selector = [
        widgets.Dropdown(
        options = [(p.name, p) for p in available],
        description = f"({i}, {j})")
    for i in range(nrows)
    for j in range(ncols)
    ]
    return background_selector

def new_foreground_selector(nrows, ncols):
    files = list((experiment_selector.value / "rois").rglob("*_rois.json"))
    foreground_selector = [
        widgets.Dropdown(
        options = [(f.name, f) for f in files],
        description = f"({i}, {j})")
    for i in range(nrows)
    for j in range(ncols)
    ]
    return foreground_selector

In [ ]:
def update_background(change, fig, axes):
    if change['type'] == 'change' and change['name'] == 'value':
        axes.cla()
        background_path = change["new"]
        if background_path.suffix == ".png":
            im = plt.imread(background_path)
            axes.imshow(im, cmap="gray")
        elif background_path.suffix == ".pkl":
            graph = nx.read_gpickle(background_path)
            attribute = list(list(graph.edges(data=True))[0][-1].keys())
            if len(attribute) != 1:
                raise ValueError(f"{background_path} has multiple edge attribute names: {attribute}")
            guplot.draw_graph_edges(fig, axes, graph, attribute[0])
        axes.set_title(background_path.name)

@widgets.interact(nrows=[1, 2, 3], ncols=[1, 2, 3])
def update(nrows=1, ncols=1):
    # erase old figure
    fig = plt.figure(1)
    plt.close(fig)
    
    # make new figure
    fig, axes = plt.subplots(nrows, ncols, clear=True, sharex=True, sharey=True, num=1, squeeze=False)
    plt.show()
    
    # make callbacks for each subplot
    bgpartials = [partial(update_background, fig=fig, axes=a) for a in axes.flat]
    
    # make selectors for each axis and attach to callbacks
    backgrounds = new_background_selector(nrows, ncols)
    for bg, bgp in zip(backgrounds, bgpartials):
        bg.observe(bgp)
    
    # group the selectors and display
    background_box = widgets.VBox([widgets.Label("backgrounds")] + backgrounds)
    foregrounds = new_foreground_selector(nrows, ncols)
    foreground_box = widgets.VBox([widgets.Label("foregrounds")] + foregrounds)
    selector_box = widgets.HBox([background_box, foreground_box])
    display(selector_box)

In [ ]:
movie_widget_list = [
    widgets.Checkbox(
        value=True,
        description=f.name,
        layout={'width': 'max-content'}
    )
    for f in (experiment_selector.value / "videos").rglob("*.h5")
]
rois_lists = list((experiment_selector.value / "rois").rglob("*_rois.json"))
rois_dict = dict()
for r in rois_lists:
    with open(r, "r") as f:
        rois_dict[r.name] = json.load(f)
movie_list = widgets.VBox(movie_widget_list)
roi_drops = [
    widgets.Dropdown(
        options=np.sort([-1] + [i["id"] for i in v]),
        description=k,
        layout={'width': 'max-content'}
    )
    for k, v in rois_dict.items()]
roi_list = widgets.VBox(roi_drops)
display(widgets.VBox(
    [movie_list, roi_list],
    layout={'width': 'max-content'}
))

In [ ]:
def plot_callback(widg):
    rois_lookup = dict()
    for roi_select in roi_drops:
        if roi_select.value != -1:
            rois_lookup[roi_select.description] = int(roi_select.value)
    rois = []
    for k, v in list(rois_lookup.items()):
        with open(experiment_selector.value / "rois" / k, "r") as f:
            j = json.load(f)
        for i in j:
            if i["id"] == v:
                rois_lookup[k] = i
    print(rois_lookup)
button = widgets.Button(
    description="plot traces",
)
button.on_click(plot_callback)
display(button)